<a href="https://colab.research.google.com/github/vickeytomer007/TextSummerization/blob/master/Final_TextSummerization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib inline
from gensim import corpora, models, similarities, matutils
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split


from sklearn.decomposition import NMF

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
news_df=pd.read_csv('/content/drive/My Drive/Colab Notebooks/data.csv')

In [4]:
news_df.head()

,news_headline,news_article,news_category
0,Dua Lipa to perform at first-ever OnePlus Musi...,International pop star Dua Lipa will perform a...,technology
1,OnePlus marks 1500 days of OxygenOS with plant...,OnePlus is celebrating 1500 days of OxygenOS a...,technology
2,Disney CEO resigns from Apple board ahead of A...,Walt Disney CEO Bob Iger resigned from Apple's...,technology
3,Google confirms its subscription service Play ...,Google took to Twitter to confirm that it's wo...,technology
4,"Indian hacker finds Uber account takeover bug,...",Uber has awarded Indian ethical hacker Anand P...,technology


In [0]:
#Tokenization.
def token(text):
    tok=[txt for txt in text.split()]
    return tok

In [0]:
news_df['headline_tok']=news_df['news_headline'].apply(lambda x:token(x))
news_df['article_tok']=news_df['news_article'].apply(lambda x:token(x))

In [7]:
#Removing Stopwords.
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords=stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
def Remove_Stopwords(text):
    words=[txt for txt in text if txt not in stopwords]
    return words

In [0]:
news_df['headline_stop']=news_df['headline_tok'].apply(lambda x:Remove_Stopwords(x))
news_df['article_stop']=news_df['article_tok'].apply(lambda x:Remove_Stopwords(x))

In [0]:
def text_lower(text):
    text=str(text)
    words="".join([txt for txt in text.lower()])
    words="".join(words)
    return words

In [0]:
news_df['article_lower']=news_df['article_stop'].apply(lambda x:text_lower(x))
news_df['headline_lower']=news_df['headline_stop'].apply(lambda x:text_lower(x))

In [0]:
def joinText(text):
    word=" ".join(text)
    return word

In [0]:
#joinText(news_df['article_stop'][0])
news_df['article_lower']=news_df['article_stop'].apply(lambda x:joinText(x))
news_df['headline_lower']=news_df['headline_stop'].apply(lambda x:joinText(x))

In [14]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
from nltk.corpus import stopwords
import time
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
print('TensorFlow Version: {}'.format(tf.__version__))

TensorFlow Version: 1.15.0


In [15]:
for i in range(5):
    print("Review #",i+1)
    print(news_df['headline_lower'][i])
    print(news_df['article_lower'][i])
    print()

Review # 1
Dua Lipa perform first-ever OnePlus Music Festival India
International pop star Dua Lipa perform first-ever OnePlus Music Festival set take place November 16 Mumbai. The festival take place DY Patil Stadium, singer Katy Perry would also performing. Tickets concert priced ₹2,000 onwards go sale 12 pm September 17.

Review # 2
OnePlus marks 1500 days OxygenOS plant tree campaign
OnePlus celebrating 1500 days OxygenOS marking occasion collaborated WWF India 'Adopt Tree' campaign. OnePlus plant one tree every tweet #OxygenOS. "It's amazing journey, building great products software together OnePlus community," said Szymon Kopec, Product lead OnePlus post Friday.

Review # 3
Disney CEO resigns Apple board ahead Apple TV+ launch
Walt Disney CEO Bob Iger resigned Apple's board directors September 10, regulatory filings revealed. This comes ahead launch Apple's recently unveiled video streaming subscription service Apple TV+, stands direct rival entertainment conglomerate's service, 

In [0]:
# A list of contractions from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [0]:
def clean_text(text, remove_stopwords = True):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    # Optionally, remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    return text

In [18]:
import nltk
nltk.download('stopwords')
  
# Clean the summaries and texts
clean_summaries = []
for summary in news_df['headline_lower']:
    clean_summaries.append(clean_text(summary, remove_stopwords=False))
print("Summaries are complete.")

clean_texts = []
for text in news_df['article_lower']:
    clean_texts.append(clean_text(text))
print("Texts are complete.")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Summaries are complete.
Texts are complete.


In [19]:
# Inspect the cleaned summaries and texts to ensure they have been cleaned well
for i in range(5):
    print("Clean Review #",i+1)
    print(news_df['headline_lower'][i])
    print(news_df['article_lower'][i])
    print()

Clean Review # 1
Dua Lipa perform first-ever OnePlus Music Festival India
International pop star Dua Lipa perform first-ever OnePlus Music Festival set take place November 16 Mumbai. The festival take place DY Patil Stadium, singer Katy Perry would also performing. Tickets concert priced ₹2,000 onwards go sale 12 pm September 17.

Clean Review # 2
OnePlus marks 1500 days OxygenOS plant tree campaign
OnePlus celebrating 1500 days OxygenOS marking occasion collaborated WWF India 'Adopt Tree' campaign. OnePlus plant one tree every tweet #OxygenOS. "It's amazing journey, building great products software together OnePlus community," said Szymon Kopec, Product lead OnePlus post Friday.

Clean Review # 3
Disney CEO resigns Apple board ahead Apple TV+ launch
Walt Disney CEO Bob Iger resigned Apple's board directors September 10, regulatory filings revealed. This comes ahead launch Apple's recently unveiled video streaming subscription service Apple TV+, stands direct rival entertainment conglo

In [0]:
def count_words(count_dict, text):
    '''Count the number of occurrences of each word in a set of text'''
    for sentence in text:
        for word in sentence.split():
            if word not in count_dict:
                count_dict[word] = 1
            else:
                count_dict[word] += 1

In [21]:
# Find the number of times each word was used and the size of the vocabulary
word_counts = {}

count_words(word_counts, clean_summaries)
count_words(word_counts, clean_texts)
            
print("Size of Vocabulary:", len(word_counts))

Size of Vocabulary: 3803


In [22]:
# Load Conceptnet Numberbatch's (CN) embeddings, similar to GloVe, but probably better 
# (https://github.com/commonsense/conceptnet-numberbatch)
embeddings_index = {}
with open('/content/drive/My Drive/Colab Notebooks/numberbatch-en-17.02.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

print('Word embeddings:', len(embeddings_index))

Word embeddings: 484557


In [23]:
# Find the number of words that are missing from CN, and are used more than our threshold.
missing_words = 0
threshold = 20

for word, count in word_counts.items():
    if count > threshold:
        if word not in embeddings_index:
            missing_words += 1
            
missing_ratio = round(missing_words/len(word_counts),4)*100
            
print("Number of words missing from CN:", missing_words)
print("Percent of words that are missing from vocabulary: {}%".format(missing_ratio))

Number of words missing from CN: 1
Percent of words that are missing from vocabulary: 0.03%


In [24]:
# Limit the vocab that we will use to words that appear ≥ threshold or are in GloVe

#dictionary to convert words to integers
vocab_to_int = {} 

value = 0
for word, count in word_counts.items():
    if count >= threshold or word in embeddings_index:
        vocab_to_int[word] = value
        value += 1

# Special tokens that will be added to our vocab
codes = ["<UNK>","<PAD>","<EOS>","<GO>"]   

# Add codes to vocab
for code in codes:
    vocab_to_int[code] = len(vocab_to_int)

# Dictionary to convert integers to words
int_to_vocab = {}
for word, value in vocab_to_int.items():
    int_to_vocab[value] = word

usage_ratio = round(len(vocab_to_int) / len(word_counts),4)*100

print("Total number of unique words:", len(word_counts))
print("Number of words we will use:", len(vocab_to_int))
print("Percent of words we will use: {}%".format(usage_ratio))

Total number of unique words: 3803
Number of words we will use: 3506
Percent of words we will use: 92.19000000000001%


In [25]:
# Need to use 300 for embedding dimensions to match CN's vectors.
embedding_dim = 300
nb_words = len(vocab_to_int)

# Create matrix with default values of zero
word_embedding_matrix = np.zeros((nb_words, embedding_dim), dtype=np.float32)
for word, i in vocab_to_int.items():
    if word in embeddings_index:
        word_embedding_matrix[i] = embeddings_index[word]
    else:
        # If word not in CN, create a random embedding for it
        new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
        embeddings_index[word] = new_embedding
        word_embedding_matrix[i] = new_embedding

# Check if value matches len(vocab_to_int)
print(len(word_embedding_matrix))

3506


In [0]:
def convert_to_ints(text, word_count, unk_count, eos=False):
    '''Convert words in text to an integer.
       If word is not in vocab_to_int, use UNK's integer.
       Total the number of words and UNKs.
       Add EOS token to the end of texts'''
    ints = []
    for sentence in text:
        sentence_ints = []
        for word in sentence.split():
            word_count += 1
            if word in vocab_to_int:
                sentence_ints.append(vocab_to_int[word])
            else:
                sentence_ints.append(vocab_to_int["<UNK>"])
                unk_count += 1
        if eos:
            sentence_ints.append(vocab_to_int["<EOS>"])
        ints.append(sentence_ints)
    return ints, word_count, unk_count

In [27]:
# Apply convert_to_ints to clean_summaries and clean_texts
word_count = 0
unk_count = 0

int_summaries, word_count, unk_count = convert_to_ints(clean_summaries, word_count, unk_count)
int_texts, word_count, unk_count = convert_to_ints(clean_texts, word_count, unk_count, eos=True)

unk_percent = round(unk_count/word_count,4)*100

print("Total number of words in headlines:", word_count)
print("Total number of UNKs in headlines:", unk_count)
print("Percent of words that are UNK: {}%".format(unk_percent))

Total number of words in headlines: 10592
Total number of UNKs in headlines: 655
Percent of words that are UNK: 6.18%


In [0]:
def create_lengths(text):
    '''Create a data frame of the sentence lengths from a text'''
    lengths = []
    for sentence in text:
        lengths.append(len(sentence))
    return pd.DataFrame(lengths, columns=['counts'])

In [29]:
lengths_summaries = create_lengths(int_summaries)
lengths_texts = create_lengths(int_texts)

print("Summaries:")
print(lengths_summaries.describe())
print()
print("Texts:")
print(lengths_texts.describe())

Summaries:
           counts
count  224.000000
mean     8.656250
std      1.530748
min      5.000000
25%      8.000000
50%      9.000000
75%     10.000000
max     13.000000

Texts:
           counts
count  224.000000
mean    39.629464
std      4.660802
min     27.000000
25%     37.000000
50%     39.000000
75%     42.000000
max     56.000000


In [30]:
# Inspect the length of texts
print(np.percentile(lengths_texts.counts, 90))
print(np.percentile(lengths_texts.counts, 95))
print(np.percentile(lengths_texts.counts, 99))

46.0
48.0
53.77000000000001


In [31]:
# Inspect the length of summaries
print(np.percentile(lengths_summaries.counts, 90))
print(np.percentile(lengths_summaries.counts, 95))
print(np.percentile(lengths_summaries.counts, 99))

11.0
11.0
12.0


In [0]:
def unk_counter(sentence):
    '''Counts the number of time UNK appears in a sentence.'''
    unk_count = 0
    for word in sentence:
        if word == vocab_to_int["<UNK>"]:
            unk_count += 1
    return unk_count

In [33]:
# takes a long time  , this is normal

# Sort the summaries and texts by the length of the texts, shortest to longest
# Limit the length of summaries and texts based on the min and max ranges.
# Remove reviews that include too many UNKs

sorted_summaries = []
sorted_texts = []
max_text_length = 84
max_summary_length = 13
min_length = 2
unk_text_limit = 1
unk_summary_limit = 0

for length in range(min(lengths_texts.counts), max_text_length): 
    for count, words in enumerate(int_summaries):
        if (len(int_summaries[count]) >= min_length and
            len(int_summaries[count]) <= max_summary_length and
            len(int_texts[count]) >= min_length and
            unk_counter(int_summaries[count]) <= unk_summary_limit and
            unk_counter(int_texts[count]) <= unk_text_limit and
            length == len(int_texts[count])
           ):
            sorted_summaries.append(int_summaries[count])
            sorted_texts.append(int_texts[count])
        
# Compare lengths to ensure they match
print(len(sorted_summaries))
print(len(sorted_texts))

84
84


In [0]:
def model_inputs():
    '''Create palceholders for inputs to the model'''
    
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    summary_length = tf.placeholder(tf.int32, (None,), name='summary_length')
    max_summary_length = tf.reduce_max(summary_length, name='max_dec_len')
    text_length = tf.placeholder(tf.int32, (None,), name='text_length')

    return input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length

In [0]:
def process_encoding_input(target_data, vocab_to_int, batch_size):
    '''Remove the last word id from each batch and concat the <GO> to the begining of each batch'''
    
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return dec_input

In [0]:
def encoding_layer(rnn_size, sequence_length, num_layers, rnn_inputs, keep_prob):
    '''Create the encoding layer'''
    
    for layer in range(num_layers):
        with tf.variable_scope('encoder_{}'.format(layer)):
            cell_fw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, 
                                                    input_keep_prob = keep_prob)

            cell_bw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, 
                                                    input_keep_prob = keep_prob)

            enc_output, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw, 
                                                                    cell_bw, 
                                                                    rnn_inputs,
                                                                    sequence_length,
                                                                    dtype=tf.float32)
    # Join outputs since we are using a bidirectional RNN
    enc_output = tf.concat(enc_output,2)
    
    return enc_output, enc_state

In [0]:
def training_decoding_layer(dec_embed_input, summary_length, dec_cell, initial_state, output_layer, 
                            vocab_size, max_summary_length):
    '''Create the training logits'''
    
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                        sequence_length=summary_length,
                                                        time_major=False)

    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                       training_helper,
                                                       initial_state,
                                                       output_layer) 

    training_logits, _ , _ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                           output_time_major=False,
                                                           impute_finished=True,
                                                           maximum_iterations=max_summary_length)
    return training_decoder

In [0]:
def inference_decoding_layer(embeddings, start_token, end_token, dec_cell, initial_state, output_layer,
                             max_summary_length, batch_size):
    '''Create the inference logits'''
    
    start_tokens = tf.tile(tf.constant([start_token], dtype=tf.int32), [batch_size], name='start_tokens')
    
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embeddings,
                                                                start_tokens,
                                                                end_token)
                
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                        inference_helper,
                                                        initial_state,
                                                        output_layer)
                
    inference_logits, _ , _ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                            output_time_major=False,
                                                            impute_finished=True,
                                                            maximum_iterations=max_summary_length)
    
    return inference_decoder

In [0]:
def decoding_layer(dec_embed_input, embeddings, enc_output, enc_state, vocab_size, text_length, summary_length, 
                   max_summary_length, rnn_size, vocab_to_int, keep_prob, batch_size, num_layers):
    '''Create the decoding cell and attention for the training and inference decoding layers'''
    
    for layer in range(num_layers):
        with tf.variable_scope('decoder_{}'.format(layer)):
            lstm = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            dec_cell = tf.contrib.rnn.DropoutWrapper(lstm, 
                                                     input_keep_prob = keep_prob)
    
    output_layer = Dense(vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    attn_mech = tf.contrib.seq2seq.BahdanauAttention(rnn_size,
                                                  enc_output,
                                                  text_length,
                                                  normalize=False,
                                                  name='BahdanauAttention')

    dec_cell = tf.contrib.seq2seq.AttentionWrapper(dec_cell,
                                                          attn_mech,
                                                          rnn_size)
            
    #initial_state = tf.contrib.seq2seq.AttentionWrapperState(enc_state[0],
    #                                                                _zero_state_tensors(rnn_size, 
    #                                                                                    batch_size, 
    #                                                                                    tf.float32)) 
    initial_state = dec_cell.zero_state(batch_size=batch_size,dtype=tf.float32).clone(cell_state=enc_state[0])

    with tf.variable_scope("decode"):
        training_decoder = training_decoding_layer(dec_embed_input, 
                                                  summary_length, 
                                                  dec_cell, 
                                                  initial_state,
                                                  output_layer,
                                                  vocab_size, 
                                                  max_summary_length)
        
        training_logits,_ ,_ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                  output_time_major=False,
                                  impute_finished=True,
                                  maximum_iterations=max_summary_length)
    with tf.variable_scope("decode", reuse=True):
        inference_decoder = inference_decoding_layer(embeddings,  
                                                    vocab_to_int['<GO>'], 
                                                    vocab_to_int['<EOS>'],
                                                    dec_cell, 
                                                    initial_state, 
                                                    output_layer,
                                                    max_summary_length,
                                                    batch_size)
        
        inference_logits,_ ,_ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                  output_time_major=False,
                                  impute_finished=True,
                                  maximum_iterations=max_summary_length)

    return training_logits, inference_logits

In [0]:
def seq2seq_model(input_data, target_data, keep_prob, text_length, summary_length, max_summary_length, 
                  vocab_size, rnn_size, num_layers, vocab_to_int, batch_size):
    '''Use the previous functions to create the training and inference logits'''
    
    # Use Numberbatch's embeddings and the newly created ones as our embeddings
    embeddings = word_embedding_matrix
    
    enc_embed_input = tf.nn.embedding_lookup(embeddings, input_data)
    enc_output, enc_state = encoding_layer(rnn_size, text_length, num_layers, enc_embed_input, keep_prob)
    
    dec_input = process_encoding_input(target_data, vocab_to_int, batch_size)
    dec_embed_input = tf.nn.embedding_lookup(embeddings, dec_input)
    
    training_logits, inference_logits  = decoding_layer(dec_embed_input, 
                                                        embeddings,
                                                        enc_output,
                                                        enc_state, 
                                                        vocab_size, 
                                                        text_length, 
                                                        summary_length, 
                                                        max_summary_length,
                                                        rnn_size, 
                                                        vocab_to_int, 
                                                        keep_prob, 
                                                        batch_size,
                                                        num_layers)
    
    return training_logits, inference_logits

In [0]:
def pad_sentence_batch(sentence_batch):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch]

In [0]:
def get_batches(summaries, texts, batch_size):
    """Batch summaries, texts, and the lengths of their sentences together"""
    for batch_i in range(0, len(texts)//batch_size):
        start_i = batch_i * batch_size
        summaries_batch = summaries[start_i:start_i + batch_size]
        texts_batch = texts[start_i:start_i + batch_size]
        pad_summaries_batch = np.array(pad_sentence_batch(summaries_batch))
        pad_texts_batch = np.array(pad_sentence_batch(texts_batch))
        
        # Need the lengths for the _lengths parameters
        pad_summaries_lengths = []
        for summary in pad_summaries_batch:
            pad_summaries_lengths.append(len(summary))
        
        pad_texts_lengths = []
        for text in pad_texts_batch:
            pad_texts_lengths.append(len(text))
        
        yield pad_summaries_batch, pad_texts_batch, pad_summaries_lengths, pad_texts_lengths

In [0]:
# Set the Hyperparameters
epochs = 100
batch_size = 40
rnn_size = 256
num_layers = 2
learning_rate = 0.005
keep_probability = 0.75

In [78]:
# Build the graph
train_graph = tf.Graph()
# Set the graph to default to ensure that it is ready for training
with train_graph.as_default():
    
    # Load the model inputs    
    input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length = model_inputs()

    # Create the training and inference logits
    training_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                      targets, 
                                                      keep_prob,   
                                                      text_length,
                                                      summary_length,
                                                      max_summary_length,
                                                      len(vocab_to_int)+1,
                                                      rnn_size, 
                                                      num_layers, 
                                                      vocab_to_int,
                                                      batch_size)
    
    # Create tensors for the training logits and inference logits
    training_logits = tf.identity(training_logits.rnn_output, 'logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')
    
    # Create the weights for sequence_loss
    masks = tf.sequence_mask(summary_length, max_summary_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(learning_rate)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)
print("Graph is built.")

Graph is built.


In [79]:
# Subset the data for training
start = 1
end = start + 1000
sorted_summaries_short = sorted_summaries[start:end]
sorted_texts_short = sorted_texts[start:end]
print("The shortest text length:", len(sorted_texts_short[0]))
print("The longest text length:",len(sorted_texts_short[-1]))

The shortest text length: 29
The longest text length: 48


In [80]:
# Train the Model
learning_rate_decay = 0.95
min_learning_rate = 0.0005
display_step = 1 # Check training loss after every 20 batches
stop_early = 0 
stop = 6 #3 # If the update loss does not decrease in 3 consecutive update checks, stop training
per_epoch = 3 # Make 3 update checks per epoch
update_check = (len(sorted_texts_short)//batch_size//per_epoch)-1

update_loss = 0 
batch_loss = 0
summary_update_loss = [] # Record the update losses for saving improvements in the model

  
tf.reset_default_graph()
checkpoint ="/content/drive/My Drive/Colab Notebooks/best_model.ckpt" #300k sentence
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    
    # If we want to continue training a previous session
    # loader = tf.train.import_meta_graph(checkpoint + '.meta')
    # loader.restore(sess, checkpoint)
    #sess.run(tf.local_variables_initializer())

    for epoch_i in range(1, epochs+1):
        update_loss = 0
        batch_loss = 0
        for batch_i, (summaries_batch, texts_batch, summaries_lengths, texts_lengths) in enumerate(
                get_batches(sorted_summaries_short, sorted_texts_short, batch_size)):
            start_time = time.time()
            _, loss = sess.run(
                [train_op, cost],
                {input_data: texts_batch,
                 targets: summaries_batch,
                 lr: learning_rate,
                 summary_length: summaries_lengths,
                 text_length: texts_lengths,
                 keep_prob: keep_probability})

            batch_loss += loss
            update_loss += loss
            end_time = time.time()
            batch_time = end_time - start_time

            if batch_i % display_step == 0 and batch_i > 0:
                print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                      .format(epoch_i,
                              epochs, 
                              batch_i, 
                              len(sorted_texts_short) // batch_size, 
                              batch_loss / display_step, 
                              batch_time*display_step))
                batch_loss = 0
                
                #saver = tf.train.Saver() 
                #saver.save(sess, checkpoint)
                
            if batch_i % update_check == 0 and batch_i > 0:
                print("Average loss for this update:", round(update_loss/update_check,3))
                summary_update_loss.append(update_loss)
                
              
                  
                # If the update loss is at a new minimum, save the model
                if update_loss <= min(summary_update_loss):
                    print('New Record!') 
                    stop_early = 0
                    saver = tf.train.Saver() 
                    saver.save(sess, checkpoint)

                else:
                    print("No Improvement.")
                    stop_early += 1
                    if stop_early == stop:
                        break
                update_loss = 0
            
                    
        # Reduce learning rate, but not below its minimum value
        learning_rate *= learning_rate_decay
        if learning_rate < min_learning_rate:
            learning_rate = min_learning_rate
        
        if stop_early == stop:
            print("Stopping Training.")
            break

Epoch   1/100 Batch    1/2 - Loss: 14.061, Seconds: 0.90
Average loss for this update: -14.061
New Record!
Epoch   2/100 Batch    1/2 - Loss: 28.572, Seconds: 0.92
Average loss for this update: -28.572
No Improvement.
Epoch   3/100 Batch    1/2 - Loss: 11.015, Seconds: 0.92
Average loss for this update: -11.015
New Record!
Epoch   4/100 Batch    1/2 - Loss:  9.418, Seconds: 0.93
Average loss for this update: -9.418
New Record!
Epoch   5/100 Batch    1/2 - Loss:  8.924, Seconds: 0.92
Average loss for this update: -8.924
New Record!
Epoch   6/100 Batch    1/2 - Loss:  8.545, Seconds: 0.90
Average loss for this update: -8.545
New Record!
Epoch   7/100 Batch    1/2 - Loss:  8.288, Seconds: 0.93
Average loss for this update: -8.288
New Record!
Epoch   8/100 Batch    1/2 - Loss:  8.002, Seconds: 0.92
Average loss for this update: -8.002
New Record!
Epoch   9/100 Batch    1/2 - Loss:  7.670, Seconds: 0.94
Average loss for this update: -7.67
New Record!
Epoch  10/100 Batch    1/2 - Loss:  7.32

In [81]:
checkpoint = "/content/drive/My Drive/Colab Notebooks/best_model.ckpt" 

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)
    names = []
    [names.append(n.name) for n in loaded_graph.as_graph_def().node]
names

INFO:tensorflow:Restoring parameters from /content/drive/My Drive/Colab Notebooks/best_model.ckpt


['input',
 'targets',
 'learning_rate',
 'keep_prob',
 'summary_length',
 'Const',
 'max_dec_len',
 'text_length',
 'ReverseV2/axis',
 'ReverseV2',
 'embedding_lookup/params_0',
 'embedding_lookup/axis',
 'embedding_lookup',
 'embedding_lookup/Identity',
 'encoder_0/DropoutWrapperInit/Const',
 'encoder_0/DropoutWrapperInit/Const_1',
 'encoder_0/DropoutWrapperInit_1/Const',
 'encoder_0/DropoutWrapperInit_1/Const_1',
 'encoder_0/bidirectional_rnn/fw/fw/Rank',
 'encoder_0/bidirectional_rnn/fw/fw/range/start',
 'encoder_0/bidirectional_rnn/fw/fw/range/delta',
 'encoder_0/bidirectional_rnn/fw/fw/range',
 'encoder_0/bidirectional_rnn/fw/fw/concat/values_0',
 'encoder_0/bidirectional_rnn/fw/fw/concat/axis',
 'encoder_0/bidirectional_rnn/fw/fw/concat',
 'encoder_0/bidirectional_rnn/fw/fw/transpose',
 'encoder_0/bidirectional_rnn/fw/fw/sequence_length',
 'encoder_0/bidirectional_rnn/fw/fw/Shape',
 'encoder_0/bidirectional_rnn/fw/fw/strided_slice/stack',
 'encoder_0/bidirectional_rnn/fw/fw/strid

In [0]:
def text_to_seq(text):
    '''Prepare the text for the model'''
    
    text = clean_text(text)
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in text.split()]

In [84]:
# Create your own review or use one from the dataset
    #input_sentence = "I have never eaten an apple before, but this red one was nice. \
                      #I think that I will try a green apple next time."
    #text = text_to_seq(input_sentence)
    random = np.random.randint(0,len(clean_texts))
    input_sentence = clean_texts[random]
    text = text_to_seq(clean_texts[random])

#    checkpoint = "best_model.ckpt"
checkpoint = "/content/drive/My Drive/Colab Notebooks/best_model.ckpt" 
    loaded_graph = tf.Graph()
    with tf.Session(graph=loaded_graph) as sess:
        # Load saved model
        loader = tf.train.import_meta_graph(checkpoint + '.meta')
        loader.restore(sess, checkpoint)

        input_data = loaded_graph.get_tensor_by_name('input:0')
        logits = loaded_graph.get_tensor_by_name('predictions:0')
        text_length = loaded_graph.get_tensor_by_name('text_length:0')
        summary_length = loaded_graph.get_tensor_by_name('summary_length:0')
        keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

        #Multiply by batch_size to match the model's input parameters
        answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                          summary_length: [np.random.randint(5,8)], 
                                          text_length: [len(text)]*batch_size,
                                          keep_prob: 1.0})[0] 

    # Remove the padding from the tweet
    pad = vocab_to_int["<PAD>"] 

    print('Original Text:', news_df['article_lower'][random])
    print('Original summary:', news_df['headline_lower'][random])#clean_summaries[random]

    print('\nText')
    print('  Word Ids:    {}'.format([i for i in text]))
    print('  Input Words: {}'.format(" ".join([int_to_vocab[i] for i in text])))

    print('\nSummary')
    print('  Word Ids:       {}'.format([i for i in answer_logits if i != pad]))
    print('  Response Words: {}'.format(" ".join([int_to_vocab[i] for i in answer_logits if i != pad])))

INFO:tensorflow:Restoring parameters from /content/drive/My Drive/Colab Notebooks/best_model.ckpt
Original Text: Sanoj Raj Bihar, became first contestant win ₹1 crore Kaun Banega Crorepati season 11, said considers money father's. "My father farmer. It's giving money him. It's money," stated. He added father complete studies due family conditions.
Original summary: This father's money: Farmer's son ₹1 crore KBC

Text
  Word Ids:    [3502, 890, 2864, 525, 3, 2865, 246, 3502, 139, 2866, 2867, 2868, 745, 88, 1227, 165, 493, 876, 876, 877, 173, 493, 493, 1754, 1496, 876, 2869, 2870, 1064, 874, 2871]
  Input Words: <UNK> raj bihar became first contestant win <UNK> crore kaun banega crorepati season 11 said considers money father father farmer giving money money stated added father complete studies due family conditions

Summary
  Word Ids:       [387, 388, 389, 390, 357, 391, 392]
  Response Words: melania criticised coat showing plane flying tower


In [85]:
from nltk.translate.bleu_score import sentence_bleu
reference = [['So', 'easy', 'so', 'good','and','all','natural']]
candidate = ['great', 'hot', 'snack']
score = sentence_bleu(reference, candidate)
print(score)

0


In [86]:
!pip install sumeval
!python -m spacy download en

     |████████████████████████████████| 81kB 3.2MB/s 
  Created wheel for sumeval: filename=sumeval-0.2.2-cp36-none-any.whl size=54535 sha256=41545e2add97d309f34c2fd2bfe3277e3bd60a3c99457b6453aac4eeecdd11ca
  Stored in directory: /root/.cache/pip/wheels/7b/6f/57/19ceecab21445c88f3c565735fa1887b4cd18d340c972eb445
Successfully built sumeval
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [87]:
#https://github.com/chakki-works/sumeval
#https://github.com/Tian312/awesome-text-summarization

from sumeval.metrics.rouge import RougeCalculator

refrence_summary = "So easy, so good and all natural"
model_summary = "great hot snack"

rouge = RougeCalculator(stopwords=True, lang="en")

rouge_1 = rouge.rouge_n(
            summary=model_summary,
            references=refrence_summary,
            n=1)

rouge_2 = rouge.rouge_n(
            summary=model_summary,
            references=[refrence_summary],
            n=2)

rouge_l = rouge.rouge_l(
            summary=model_summary,
            references=[refrence_summary])

# You need spaCy to calculate ROUGE-BE

rouge_be = rouge.rouge_be(
            summary=model_summary,
            references=[refrence_summary])

print("ROUGE-1: {}, ROUGE-2: {}, ROUGE-L: {}, ROUGE-BE: {}".format(
    rouge_1, rouge_2, rouge_l, rouge_be
).replace(", ", "\n"))

b.great=(amod)=>snack
b.hot=(amod)=>snack
ROUGE-1: 0
ROUGE-2: 0
ROUGE-L: 0
ROUGE-BE: 0


In [88]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()

sentence = ("""My little Chihuahua and Jack Russell mix just loves these! She 
gobbles them up! They are a perfect
size for her small mouth. According
to her, they are very tasty.""","")
model = ("My Dog Loves Them!","")
summary = ("Chihuahua mix perfect","")

tfidf_matrix_sentence = tfidf_vectorizer.fit_transform(sentence) 
tfidf_matrix_model = tfidf_vectorizer.transform(model) 
tfidf_matrix_summary = tfidf_vectorizer.transform(summary) 


cosine = cosine_similarity(tfidf_matrix_sentence, tfidf_matrix_model)
print(cosine)

cosine = cosine_similarity(tfidf_matrix_sentence, tfidf_matrix_summary)
print(cosine)

[[0.29277002 0.        ]
 [0.         0.        ]]
[[0.29277002 0.        ]
 [0.         0.        ]]
